# Data Wrangling Project: OpenStreetMap Brooklyn Data Case Study

## Map area

Area of study:
Brooklyn, NY, United States

https://mapzen.com/data/metro-extracts/metro/brooklyn_new-york/
I just moved my daughter into a small apartment here. I know it's a densely packed area with lots of interesting features.

## Problems in the data

Problems encountered in the data include the following:

*   Overabbrieviated street names
*   Unrecognized or miscatagorized street names
*   Ill-formatted postal codes
*   Postcodes outside of Brooklyn

## Cleaning street name data

The code shown here is in the attached data files:

*   data_final.py: Parse the XML data, prepare for the SQL database, and write to CSV.



In [ ]:
import xml.etree.cElementTree as ET
import pprint
import re
from collections import defaultdict

Some regular expressions to search for common problems.

In [38]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

Downloaded data files for processing.  

Source: https://mapzen.com/data/metro-extracts/metro/brooklyn_new-york/



In [39]:
OSMFILE = '/Users/gdimino/python/osm_files/brooklyn_new-york.osm'
SAMPLE_FILE = '/Users/gdimino/python/osm_files/brooklyn_sample_k_100.osm'

Next, some code that iterates throught the xml elements and check the k attribute (street) for the common problems in defined in the regexes.

In [40]:
def key_type(element, keys):
    if element.tag == "tag":
        # NEW CODE HERE
        if (re.search(lower, element.attrib['k'])):
            keys['lower'] = keys['lower'] + 1
        elif (re.search(lower_colon, element.attrib['k'])):
            keys['lower_colon'] = keys['lower_colon'] + 1
        elif (re.search(problemchars, element.attrib['k'])):
            keys['problemchars'] = keys['problemchars'] + 1
        else:
            keys['other'] = keys['other'] + 1
    return keys

def iterparse_with_error(filename, xml):
    while True:
        try:
            yield next(ET.iterparse(filename))
        except StopIteration:
            raise
        except xml.parsers.expat.ExpatError, ex:
            print "excpetion: %s" % (ex)
            continue

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
   # for _, element in iterparse_with_error(filename):
    for _, element in ET.iterparse(filename):

        keys = key_type(element, keys)

    return keys

Now, we'll run the audit code on the sample data, to 

In [41]:
my_map = process_map(SAMPLE_FILE)


In [42]:
my_map



{'lower': 10445, 'lower_colon': 17479, 'other': 68, 'problemchars': 142}

In [43]:
def count_tags(filename):
        # New code here
        mytags = {}
        for event, el in ET.iterparse(filename):
             if el.tag in mytags:
                 mytags[el.tag] = mytags[el.tag] + 1
             else:
                 mytags[el.tag] = 1
        return mytags

In [44]:
my_tags = count_tags(SAMPLE_FILE)

In [45]:
my_tags


{'member': 311,
 'nd': 35138,
 'node': 24852,
 'osm': 1,
 'relation': 17,
 'tag': 28134,
 'way': 4904}

In [46]:
def unique_users(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if element.attrib.has_key('uid'):
          users.add(element.attrib['uid'])
    return users

In [47]:
user_set = unique_users(SAMPLE_FILE)

In [48]:
user_set

{'100643',
 '1006495',
 '102691',
 '1068690',
 '1072240',
 '108704',
 '109276',
 '1093326',
 '109362',
 '1110270',
 '113450',
 '11617',
 '119748',
 '119881',
 '120468',
 '12055',
 '120567',
 '123364',
 '1240849',
 '1263353',
 '127',
 '129614',
 '130472',
 '1306',
 '1311287',
 '13203',
 '134892',
 '1361318',
 '1361320',
 '1361327',
 '1376118',
 '1380816',
 '1403907',
 '1424765',
 '14293',
 '147510',
 '147851',
 '148169',
 '1494110',
 '153669',
 '1556219',
 '158826',
 '1611',
 '1624486',
 '165061',
 '165869',
 '1672350',
 '1679',
 '17068',
 '1714546',
 '1721804',
 '1732559',
 '1751737',
 '1758669',
 '1758854',
 '1764427',
 '1764514',
 '1769120',
 '1769121',
 '1772879',
 '1772882',
 '1772885',
 '1772886',
 '1772887',
 '1772888',
 '1772889',
 '1772890',
 '1772893',
 '1772901',
 '1772905',
 '1772933',
 '1773157',
 '1778799',
 '1781102',
 '1781294',
 '178186',
 '1782960',
 '1795644',
 '1812841',
 '1815736',
 '1816453',
 '1829683',
 '1851008',
 '1866507',
 '1868866',
 '1876172',
 '1886879',
 

In [49]:
len(user_set)

292

In [50]:
full_user_set = unique_users(OSMFILE)

In [51]:
full_user_set

{'1361332',
 '2109958',
 '2406578',
 '461644',
 '4260227',
 '93788',
 '3330978',
 '576515',
 '1586627',
 '2176208',
 '239918',
 '2411240',
 '73269',
 '340102',
 '137124',
 '1829299',
 '2642214',
 '674438',
 '904598',
 '4732',
 '3397394',
 '2309492',
 '579990',
 '4160838',
 '1072240',
 '210173',
 '118021',
 '2163609',
 '2815098',
 '2808681',
 '2027261',
 '336460',
 '2414553',
 '2414554',
 '323054',
 '4085543',
 '128470',
 '927001',
 '408282',
 '537631',
 '1651149',
 '1795637',
 '610611',
 '1795639',
 '21927',
 '11617',
 '1851008',
 '2267074',
 '618377',
 '54552',
 '4141287',
 '3407412',
 '2644570',
 '622545',
 '346758',
 '2252969',
 '42123',
 '479256',
 '419214',
 '71823',
 '2820500',
 '123364',
 '459441',
 '2755657',
 '2481351',
 '510836',
 '1872273',
 '3339202',
 '989421',
 '45062',
 '2795656',
 '2644101',
 '2329944',
 '2255358',
 '2074114',
 '434368',
 '1876172',
 '779673',
 '23500',
 '168517',
 '651621',
 '3677802',
 '123633',
 '68540',
 '338611',
 '1042877',
 '2406156',
 '2725926',

In [52]:
len(full_user_set)

1410

In [53]:
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# expected = expected + ["Expressway", "Terrace"]

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
    else:
        street_type = 'ERROR'
 #   print street_type
    
    
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

In [54]:
odd_streets = audit(SAMPLE_FILE)

In [55]:
odd_streets


defaultdict(set,
            {'A': {'Avenue A'},
             'Americas': {'Avenue Of The Americas'},
             'B': {'Avenue B'},
             'Bayside': {'Bayside'},
             'Bowery': {'Bowery'},
             'Broadway': {'Broadway',
              'East Broadway',
              'W. Broadway',
              'West Broadway'},
             'C': {'Avenue C'},
             'Crescent': {'Boelsen Crescent',
              'Ellwell Crescent',
              'Mt Olivet Crescent'},
             'D': {'Avenue D'},
             'East': {'Union Square East'},
             'F': {'Avenue F'},
             'Finest': {'Avenue Of The Finest'},
             'H': {'Avenue H'},
             'Hamilton': {'Fort Hamilton'},
             'Highway': {'Kings Highway'},
             'I': {'Avenue I'},
             'J': {'Avenue J'},
             'K': {'Avenue K'},
             'L': {'Avenue L'},
             'M': {'Avenue M'},
             'N': {'Avenue N'},
             'North': {'Barlow Drive North',
  

This gives some good ideas of things to add to expected: Walk, Way, Turnpike, Terrace, Plaza, Park, Esplenade,Concourse, Mews

In [57]:
expected = expected + ['Concourse', 'Esplenade', 'Mews', 'Park', 'Plaza', 'Turnpike', 'Walk', 'Way']

In [58]:
full_street_types = audit(OSMFILE)

In [63]:
full_street_types

defaultdict(set,
            {'1': {'Graham Avenue #1'},
             '109': {'Avenue of the Americas #109'},
             '11217': {'305 Schermerhorn St., Brooklyn, NY 11217'},
             '1st': {'1st'},
             '218650358': {'218650358'},
             '3': {'Hanover Square #3'},
             '300': {'Ste 300'},
             '4B': {'Union Avenue 4B'},
             '506': {'Broadway #506'},
             '861': {'861'},
             'A': {'Avenue A'},
             'Alley': {'Cortlandt Alley',
              'Exchange Alley',
              'Freeman Alley',
              'Grace Court Alley',
              'Harrison Alley',
              'Mac Dougal Alley',
              'Theater Alley'},
             'Americas': {'Avenue Of The Americas',
              'Avenue of the Americas',
              'Avenue of the Americas\n'},
             'Atrium': {'Broadway Atrium'},
             'Ave': {'4th Ave',
              '5th Ave',
              '6th Ave',
              'Norman Ave',
           

Now, we can fix some of the common problems.  Let's create a mapping to fix them, and then a function to apply the fix.  We also notice that some street names end it punctuation, so let's strip that.

## Building the mySQL database


SQL for building the tables:
https://gist.github.com/swwelch/f1144229848b407e0a5d13fcb7fbbd6f

Sample MySQL command for loading the CSVs into the tables:

`mysql> load data local infile '/Users/gdimino/python/osm_files/nodes_tags.csv' into table nodes_tags fields terminated by ',';`


Problems:
*   data.py wrote files in dos format.
*   forgot to remove the first line.

Fixed the problems in the source files and reran the command.

Now, let's check that we have all the tables we need and get ready to query the database.

```
mysql> show tables;

+-------------------------+

| Tables_in_brooklyn_full |

+-------------------------+

| nodes                   |

| nodes_tags              |

| ways                    |

| ways_nodes              |

| ways_tags               |

+-------------------------+

5 rows in set (0.00 sec)
```

## Postal codes

### Looking at postcodes

Here's a query of postcodes ordered by frequency:


```
mysql> SELECT tags.value, COUNT(*) as count 
FROM (SELECT * FROM nodes_tags 
UNION ALL 
SELECT * FROM ways_tags) tags
WHERE tags.key='postcode'
GROUP BY tags.value
ORDER BY count DESC;

+----------------+-------+
| value          | count |
+----------------+-------+
| 11234          | 20139 |
| 11236          | 15226 |
| 11385          | 15091 |
| 11229          | 12542 |
| 11203          | 11773 |
| 11207          | 11176 |
| 11208          | 11131 |
| 11223          | 10756 |
| 11214          | 10052 |
| 11204          |  9897 |
| 11215          |  9565 |
| 11219          |  9474 |
| 11221          |  9397 |
| 11210          |  9322 |
| 11209          |  9212 |
| 11220          |  9186 |
| 11379          |  8708 |
| 11235          |  8616 |
| 11228          |  8593 |
| 11230          |  8506 |
| 11378          |  8132 |
| 11233          |  8010 |
| 11375          |  7684 |
| 11218          |  7230 |
| 11212          |  6753 |
| 11421          |  6382 |
| 11216          |  6381 |
| 11211          |  6376 |
| 11226          |  6144 |
| 11222          |  6117 |
| 11417          |  5569 |
| 11206          |  5478 |
| 11213          |  5314 |
| 11414          |  4918 |
| 11231          |  4916 |
| 11238          |  4696 |
| 11237          |  4566 |
| 11217          |  4522 |
| 11416          |  4135 |
| 11225          |  4099 |
| 11201          |  3942 |
| 11694          |  3461 |
| 11374          |  3428 |
| 11418          |  3351 |
| 11373          |  3307 |
| 11205          |  3248 |
| 11232          |  3216 |
| 11224          |  2831 |
| 11697          |  2742 |
| 11377          |  2520 |
| 10003          |  2503 |
| 10002          |  2421 |
| 10014          |  2375 |
| 10013          |  2139 |
| 11249          |  2007 |
| 10009          |  1661 |
| 10012          |  1533 |
| 10011          |   943 |
| 10038          |   643 |
| 11419          |   539 |
| 11368          |   480 |
| 11101          |   399 |
| 10007          |   343 |
| 10010          |   336 |
| 10004          |   313 |
| 11104          |   295 |
| 10005          |   206 |
| 11239          |   150 |
| 11415          |   148 |
| 10006          |   131 |
| 07302          |    61 |
| 10280          |    37 |
| 10282          |    36 |
| 11367          |    14 |
| 10281          |     9 |
| 07030          |     9 |
| 10001          |     8 |
| 11224-4003     |     5 |
| 10048          |     4 |
| 11251          |     3 |
| 07310          |     3 |
| 10275          |     2 |
| 07311          |     2 |
| 10016          |     2 |
| 11420          |     2 |
| 11201-2483     |     1 |
| 07030-5774     |     1 |
| NY 11201       |     1 |
| 10011-6832     |     1 |
| NY 10002       |     1 |
| NY 11221       |     1 |
| 10012-3332     |     1 |
| (718) 778-0140 |     1 |
| 10018          |     1 |
| NY 10003       |     1 |
| 1011           |     1 |
| 11215-9993     |     1 |
| 11201;11231    |     1 |
| 11231;11230    |     1 |
| 11214;11223    |     1 |
| 11232-2400     |     1 |
| 10023          |     1 |
| 10002-1013     |     1 |
+----------------+-------+
103 rows in set (25.03 sec)
```

#### Problems
You can see quite a few problems, and opportunities for further data cleansing.  For example:

   * The state name in the postcode field.
   * A telephone number in the postcode field
   
There is a further anomaly: Not all of those postal codes are in Brooklyn!  Some are in Manhattan (start w/ 10 instead of 11). Some are even in NJ (start 07)!

Looking at the zipcode map: https://www.unitedstateszipcodes.org/ny/

...you can see that if you draw a rectangle that contains all of Brooklyn, you will end up including bits of lower Manhattan and even part of Jersey City.  Since those are the zipcodes that are included here, I am postulating that something similar accounts for these postcodes being in the Brooklyn dataset.

But, for a rectangle, you'd expect to see more of those postcodes than you do, so I'm guessing it is a polygon with more sides, taking in smaller slivers of Manhattan and Jersey City.

So, here's another opportunity to improve the data--remove the nodes that are not actually in Brooklyn!


### Top amenities


```
mysql> select value, count(*) as num  from nodes_tags where `key`='amenity' group by value order by num desc limit 25;
mysql> SELECT value, COUNT(*) as num
  FROM nodes_tags
  WHERE `key`='amenity'
  GROUP BY value
  ORDER BY num DESC
  LIMIT 10;

+------------------+------+
| value            | num  |
+------------------+------+
| bicycle_parking  | 2817 |
| restaurant       |  858 |
| place_of_worship |  366 |
| school           |  360 |
| cafe             |  286 |
| bench            |  263 |
| bicycle_rental   |  261 |
| bar              |  176 |
| fast_food        |  175 |
| fire_station     |  121 |
+------------------+------+
10 rows in set (0.25 sec)
```

### Restaurants by type

```
mysql> SELECT nodes_tags.value, COUNT(*) as num 
FROM nodes_tags      
JOIN (SELECT(id) FROM nodes_tags WHERE value='restaurant') i     
ON nodes_tags.id=i.id WHERE nodes_tags.key='cuisine' 
GROUP BY nodes_tags.value 
ORDER BY num DESC 
LIMIT 20;

+----------------------------------------------+-------+
| value                                        | num   |
+----------------------------------------------+-------+
| pizza                                        | 42292 |
| coffee_shop                                  | 39368 |
| mexican                                      | 34998 |
| american                                     | 32087 |
| burger                                       | 29889 |
| italian                                      | 29168 |
| chinese                                      | 24061 |
| sandwich                                     | 19684 |
| french                                       | 14584 |
| thai                                         | 13123 |
| sushi                                        | 11665 |
| japanese                                     | 10939 |
| indian                                       |  8749 |
| vietnamese                                   |  7290 |
| asian                                        |  6562 |
| russian                                      |  6561 |
| donut                                        |  5832 |
| chicken                                      |  5104 |
| ice_cream                                    |  5103 |
| barbecue                                     |  5103 |
+----------------------------------------------+-------+
```

### Places of worship by religion

```
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship') i
ON nodes_tags.id=i.id
WHERE nodes_tags.key='religion'
GROUP BY nodes_tags.value
ORDER BY num DESC;

+-----------+-----+
| value     | num |
+-----------+-----+
| christian | 293 |
| jewish    |  45 |
| muslim    |   7 |
| buddhist  |   4 |
| hindu     |   3 |
+-----------+-----+
```

### What types of religious schools are represented?


```
mysql> SELECT nodes_tags.value, COUNT(*) as num
    -> FROM nodes_tags 
    ->     JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='school') i
    ->     ON nodes_tags.id=i.id
    -> WHERE nodes_tags.key='religion'
    -> GROUP BY nodes_tags.value
    -> ORDER BY num DESC;
+-----------+-----+
| value     | num |
+-----------+-----+
| christian |  15 |
| jewish    |   4 |
| hindu     |   3 |
| muslim    |   3 |
+-----------+-----+
4 rows in set (0.44 sec)
```

Most 